
<h3>导入相关库<h3/>

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb

D:\Tool\Anaconda\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


<h3>所用到的相关变量以及参数<h3/>

In [2]:
city = ['fuyang','gongshu','linan','linpingqu','qiantangqu','shangcheng','xiaoshan','xihu','yuhang']
filterColumn = ['小区名称','区域位置','总价','单价','关注度','所在楼层','建筑面积','户型结构','建筑类型','房屋朝向','建筑结构','装修情况','配备电梯','交易权属','房屋用途','房屋年限','产权所属','抵押信息']

<h3>将多个csv进行连接(处理用作绘制相关图标以及查询小区基本信息)<h3/>

In [3]:
result = pd.read_csv('../data/binjiang杭州二手房.csv',encoding='GB18030',index_col=0)
for i in city:
    data = pd.read_csv('../data/{}杭州二手房.csv'.format(i),encoding='GB18030',index_col=0)
    result = pd.concat([result,data],ignore_index=True)
result

,小区名称,区域位置,经度,纬度,总价,单价,看房时间,链家编号,关注度,房屋户型,...,用水类型,用电类型,燃气价格,户型介绍,适宜人群,装修描述,售房详情,交通出行,别墅类型,权属抵押
0,['欣盛东方郡西区'],滨江-滨江区政府,120.218825,30.203884,930.0,69053,提前预约随时可看,103117810968,4,3室2厅1厨2卫,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,['铂金时代公寓'],滨江-滨江区政府,120.223773,30.201048,765.0,55423,下班后可看,103117801479,7,3室2厅1厨2卫,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['星耀城'],滨江-滨江区政府,120.221367,30.219193,225.0,34611,提前预约随时可看,103108527621,104,3室1厅1厨2卫,...,商水,商电,3.1元/m3,\n 本房地理位置优越，高品质，开发商精装，适合年轻上...,\n 地铁1号线和6号线江陵站，比邻江南大道，过三桥、...,NaN,NaN,NaN,NaN,NaN
3,['中赢康康谷'],滨江-滨江区政府,120.220640,30.202140,148.0,31835,提前预约随时可看,103117403346,18,2室2厅1厨2卫,...,民水,民电,3.1元/m3,NaN,NaN,\n \n小区附 近500米为滨和路地铁口，出行便利，...,NaN,NaN,NaN,NaN
4,['玲珑府'],滨江-西兴,120.232076,30.189648,130.0,26461,提前预约随时可看,103117598758,11,2室1厅1厨2卫,...,商水,民电,3.1元/m3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29022,['良渚文化村七贤郡'],余杭-良渚,120.007622,30.353357,280.0,31743,提前预约随时可看,103116072249,1,3室2厅1厨1卫,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29023,['西城时代家园'],余杭-未来科技城,119.958771,30.271248,325.0,35342,提前预约随时可看,103114913580,13,3室2厅1厨1卫,...,NaN,NaN,NaN,NaN,NaN,NaN,\n 小区1公里内有幼儿园 学 初中 万达广场 江...,NaN,NaN,NaN
29024,['众安学君里'],余杭-未来科技城,119.969694,30.252949,295.0,32859,提前预约随时可看,103112976063,22,3室2厅1厨1卫,...,NaN,NaN,NaN,NaN,NaN,\n \n小区位于天目山西路与东西大道交界处，沿东西大...,NaN,NaN,NaN,NaN
29025,['北秀蓝湾花苑'],余杭-良渚,120.054093,30.373145,320.0,25211,提前预约随时可看,103113428700,10,3室2厅1厨2卫,...,NaN,NaN,NaN,\n 2号线良渚站 杜甫站均1.2公里左右 ，直线距离...,NaN,NaN,NaN,NaN,NaN,NaN


<h3>筛选字段<h3/>

In [4]:
select = result[filterColumn]
select

,小区名称,区域位置,总价,单价,关注度,所在楼层,建筑面积,户型结构,建筑类型,房屋朝向,建筑结构,装修情况,配备电梯,交易权属,房屋用途,房屋年限,产权所属,抵押信息
0,['欣盛东方郡西区'],滨江-滨江区政府,930.0,69053,4,中楼层 (共18层),134.68㎡,平层,板塔结合,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,共有,\n 无抵押\n ...
1,['铂金时代公寓'],滨江-滨江区政府,765.0,55423,7,低楼层 (共30层),138.03㎡,平层,板塔结合,南 北,砖混结构,精装,有,商品房,普通住宅,满五年,共有,\n 有抵押 业主自还\n ...
2,['星耀城'],滨江-滨江区政府,225.0,34611,104,低楼层 (共20层),65.01㎡,暂无数据,板塔结合,西南,钢混结构,精装,有,商品房,商住两用,满五年,共有,\n 无抵押\n ...
3,['中赢康康谷'],滨江-滨江区政府,148.0,31835,18,低楼层 (共18层),46.49㎡,暂无数据,板楼,南,钢混结构,精装,有,商品房,商住两用,满两年,共有,\n 无抵押\n ...
4,['玲珑府'],滨江-西兴,130.0,26461,11,中楼层 (共21层),49.13㎡,暂无数据,板塔结合,南,钢混结构,精装,有,商品房,商住两用,满两年,非共有,\n 有抵押 34万元 招商银...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29022,['良渚文化村七贤郡'],余杭-良渚,280.0,31743,1,高楼层 (共9层),88.21㎡,平层,板楼,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,非共有,\n 有抵押 40万元 汇丰银...
29023,['西城时代家园'],余杭-未来科技城,325.0,35342,13,中楼层 (共11层),91.96㎡,平层,板楼,南,钢混结构,简装,有,商品房,普通住宅,满两年,非共有,\n 有抵押\n ...
29024,['众安学君里'],余杭-未来科技城,295.0,32859,22,高楼层 (共24层),89.78㎡,平层,板塔结合,南 北,钢混结构,简装,有,商品房,普通住宅,满两年,共有,\n 有抵押 80万元 业主自...
29025,['北秀蓝湾花苑'],余杭-良渚,320.0,25211,10,中楼层 (共11层),126.93㎡,平层,板楼,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,共有,\n 无抵押\n ...


<h3>填充缺失值<h3/>

In [5]:
select['户型结构'].fillna(select['户型结构'].mode()[0],inplace=True)
select['建筑类型'].fillna(select['建筑类型'].mode()[0],inplace=True)
select['建筑结构'].fillna(select['建筑结构'].mode()[0],inplace=True)
select['装修情况'].fillna(select['装修情况'].mode()[0],inplace=True)
select['配备电梯'].fillna(select['配备电梯'].mode()[0],inplace=True)

C:\Users\Shirin\AppData\Local\Temp\ipykernel_11832\1472428159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select['户型结构'].fillna(select['户型结构'].mode()[0],inplace=True)
C:\Users\Shirin\AppData\Local\Temp\ipykernel_11832\1472428159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select['建筑类型'].fillna(select['建筑类型'].mode()[0],inplace=True)
C:\Users\Shirin\AppData\Local\Temp\ipykernel_11832\1472428159.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

<h3>截取字段<h3/>

In [6]:
select['小区名称'] = select['小区名称'].str[2:-2]
select['所在楼层'] = select['所在楼层'].str[0]
select['建筑面积'] = select['建筑面积'].str[0:-1]
def extract_pledged(row):
    if re.search(r'无抵押', row):
        return '无'
    elif re.search(r'有抵押', row):
        return '有'
    else:
        return '未知'
select[['城市','街道']] = select['区域位置'].str.split('-',expand=True)
select.drop(columns='区域位置',inplace=True)

C:\Users\Shirin\AppData\Local\Temp\ipykernel_11832\2951482925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select['小区名称'] = select['小区名称'].str[2:-2]
C:\Users\Shirin\AppData\Local\Temp\ipykernel_11832\2951482925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select['所在楼层'] = select['所在楼层'].str[0]
C:\Users\Shirin\AppData\Local\Temp\ipykernel_11832\2951482925.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

<h3>保存<h3/>

In [7]:
select['建筑面积'] = select['建筑面积'].astype(float)
select.to_csv('../data/deal/total_fordraw.csv')

C:\Users\Shirin\AppData\Local\Temp\ipykernel_11832\3190850317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select['建筑面积'] = select['建筑面积'].astype(float)


### 重新读取信息(处理用作数据历史数据查询)

In [54]:
result = pd.read_csv('../data/binjiang杭州二手房.csv',encoding='GB18030',index_col=0)
for i in city:
    data = pd.read_csv('../data/{}杭州二手房.csv'.format(i),encoding='GB18030',index_col=0)
    result = pd.concat([result,data],ignore_index=True)
result = result[filterColumn]
result

,小区名称,区域位置,总价,单价,关注度,所在楼层,建筑面积,户型结构,建筑类型,房屋朝向,建筑结构,装修情况,配备电梯,交易权属,房屋用途,房屋年限,产权所属,抵押信息
0,['欣盛东方郡西区'],滨江-滨江区政府,930.0,69053,4,中楼层 (共18层),134.68㎡,平层,板塔结合,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,共有,\n 无抵押\n ...
1,['铂金时代公寓'],滨江-滨江区政府,765.0,55423,7,低楼层 (共30层),138.03㎡,平层,板塔结合,南 北,砖混结构,精装,有,商品房,普通住宅,满五年,共有,\n 有抵押 业主自还\n ...
2,['星耀城'],滨江-滨江区政府,225.0,34611,104,低楼层 (共20层),65.01㎡,暂无数据,板塔结合,西南,钢混结构,精装,有,商品房,商住两用,满五年,共有,\n 无抵押\n ...
3,['中赢康康谷'],滨江-滨江区政府,148.0,31835,18,低楼层 (共18层),46.49㎡,暂无数据,板楼,南,钢混结构,精装,有,商品房,商住两用,满两年,共有,\n 无抵押\n ...
4,['玲珑府'],滨江-西兴,130.0,26461,11,中楼层 (共21层),49.13㎡,暂无数据,板塔结合,南,钢混结构,精装,有,商品房,商住两用,满两年,非共有,\n 有抵押 34万元 招商银...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29022,['良渚文化村七贤郡'],余杭-良渚,280.0,31743,1,高楼层 (共9层),88.21㎡,平层,板楼,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,非共有,\n 有抵押 40万元 汇丰银...
29023,['西城时代家园'],余杭-未来科技城,325.0,35342,13,中楼层 (共11层),91.96㎡,平层,板楼,南,钢混结构,简装,有,商品房,普通住宅,满两年,非共有,\n 有抵押\n ...
29024,['众安学君里'],余杭-未来科技城,295.0,32859,22,高楼层 (共24层),89.78㎡,平层,板塔结合,南 北,钢混结构,简装,有,商品房,普通住宅,满两年,共有,\n 有抵押 80万元 业主自...
29025,['北秀蓝湾花苑'],余杭-良渚,320.0,25211,10,中楼层 (共11层),126.93㎡,平层,板楼,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,共有,\n 无抵押\n ...


### 处理抵押信息

In [55]:
def extract_numbers(s):
    # 使用正则表达式查找所有数字
    numbers = re.findall(r'\d+', s)
    # 如果找到了数字，则返回第一个找到的数字（假设每个字符串只有一个数字）
    # 如果需要处理多个数字，可以根据需要修改这里的逻辑
    if numbers:
        return ',' + numbers[0] + 'w'  # 转换为整数
    else:
        return None  # 如果没有找到数字，返回None或其他默认值
def extract_pledged(row):
    if re.search(r'无抵押', row):
        return '无'
    elif re.search(r'有抵押', row):
        return '有'
    else:
        return '未知'
result['欠款'] = result['抵押信息'].apply(extract_numbers)
result['抵押信息'] = result['抵押信息'].apply(extract_pledged)
result['抵押信息'] = result['抵押信息']+result['欠款']
result['抵押信息'].fillna('无',inplace=True)

### 处理缺失值

In [56]:
result['户型结构'].fillna(result['户型结构'].mode()[0],inplace=True)
result['建筑类型'].fillna(result['建筑类型'].mode()[0],inplace=True)
result['建筑结构'].fillna(result['建筑结构'].mode()[0],inplace=True)
result['装修情况'].fillna(result['装修情况'].mode()[0],inplace=True)
result['配备电梯'].fillna(result['配备电梯'].mode()[0],inplace=True)
result

,小区名称,区域位置,总价,单价,关注度,所在楼层,建筑面积,户型结构,建筑类型,房屋朝向,建筑结构,装修情况,配备电梯,交易权属,房屋用途,房屋年限,产权所属,抵押信息,欠款
0,['欣盛东方郡西区'],滨江-滨江区政府,930.0,69053,4,中楼层 (共18层),134.68㎡,平层,板塔结合,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,共有,无,None
1,['铂金时代公寓'],滨江-滨江区政府,765.0,55423,7,低楼层 (共30层),138.03㎡,平层,板塔结合,南 北,砖混结构,精装,有,商品房,普通住宅,满五年,共有,无,None
2,['星耀城'],滨江-滨江区政府,225.0,34611,104,低楼层 (共20层),65.01㎡,暂无数据,板塔结合,西南,钢混结构,精装,有,商品房,商住两用,满五年,共有,无,None
3,['中赢康康谷'],滨江-滨江区政府,148.0,31835,18,低楼层 (共18层),46.49㎡,暂无数据,板楼,南,钢混结构,精装,有,商品房,商住两用,满两年,共有,无,None
4,['玲珑府'],滨江-西兴,130.0,26461,11,中楼层 (共21层),49.13㎡,暂无数据,板塔结合,南,钢混结构,精装,有,商品房,商住两用,满两年,非共有,"有,34w",",34w"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29022,['良渚文化村七贤郡'],余杭-良渚,280.0,31743,1,高楼层 (共9层),88.21㎡,平层,板楼,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,非共有,"有,40w",",40w"
29023,['西城时代家园'],余杭-未来科技城,325.0,35342,13,中楼层 (共11层),91.96㎡,平层,板楼,南,钢混结构,简装,有,商品房,普通住宅,满两年,非共有,无,None
29024,['众安学君里'],余杭-未来科技城,295.0,32859,22,高楼层 (共24层),89.78㎡,平层,板塔结合,南 北,钢混结构,简装,有,商品房,普通住宅,满两年,共有,"有,80w",",80w"
29025,['北秀蓝湾花苑'],余杭-良渚,320.0,25211,10,中楼层 (共11层),126.93㎡,平层,板楼,南 北,钢混结构,精装,有,商品房,普通住宅,满五年,共有,无,None


### 截取字段

In [57]:
result['小区名称'] = result['小区名称'].str[2:-2]
result['建筑面积'] = result['建筑面积'].str[0:-1]
def extract_pledged(row):
    if re.search(r'无抵押', row):
        return '无'
    elif re.search(r'有抵押', row):
        return '有'
    else:
        return '未知'
result[['城市','街道']] = result['区域位置'].str.split('-',expand=True)
result.drop(columns='区域位置',inplace=True)
result.drop(columns='欠款',inplace=True)
result.to_csv('../data/deal/total_forhistory.csv')